Finally, we write the code file itself.  Most of the text below will appear literally in the file.  Statements in single curly braces will be substituted by the `format` statement at the end of the string; double curly braces will just appear as literal braces in the output.

In the following, the ODE system directly integrates the quaternion rotors describing the spin directions and orbital angular velocity vector.  This allows us to reduce the number of variables in the system and automatically satisfy certain constraints, which should lead to better numerical behavior.

Note that in this formulation, `Phi` is a completely unnecessary variable.  It doesn't need to be computed because it is implict in the definition of the frame.  It is calculated here simply because it is easy to do, and might be handy to have around.  Of course, it could be derived from knowledge of the frame alone.

In [1]:
# Always run this first
# NOTE: Do not define new basic variables in this notebook;
#       define them in Variables_Q.ipynb.  Use this notebook
#       to define new expressions built from those variables.

from __future__ import division # This needs to be here, even though it's in Variables.py
import sys
sys.path.insert(0, '..') # Look for modules in directory above this one
execfile('../Utilities/ExecNotebook.ipy')
UseQuaternions = True
from sympy import N
from sympy import Rational as frac # Rename for similarity to latex
execnotebook('../PNTerms/Variables_Q.ipynb')
from Utilities import CodeOutput
from textwrap import indent

/home/dongze/.local/lib/python3.8/site-packages/IPython/nbformat.py:12: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  warn("The `IPython.nbformat` package has been deprecated since IPython 4.0. "
/home/dongze/.local/lib/python3.8/site-packages/nbformat/current.py:15: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  warnings.warn("""nbformat.current is deprecated.


In [5]:
with open('PNEvolution.py', 'w') as f :
    f.write("""# File produced automatically by PNCodeGen.ipynb
from scipy.integrate import solve_ivp
import numpy as np
from numpy import conjugate, dot, exp, log, sqrt, pi
from numpy import euler_gamma as EulerGamma
from scipy.interpolate import InterpolatedUnivariateSpline as Spline
import quaternion
""")

    for PNOrbitalEvolutionOrder in [frac(n,2) for n in range(0,13)]:
        print("Working on {0} PN...".format(PNOrbitalEvolutionOrder))
        PNOrbitalEvolutionOrderString = str(N(PNOrbitalEvolutionOrder,2)).replace('.','p')
        %cd -q ../PNTerms
        execnotebook('OrbitalEvolution.ipynb')
        ExpressionsForMemberFunctions = CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities)
        execnotebook('AngularMomentum.ipynb')
        %cd -q -
        PrecessionVelocities.AddDerivedVariable('OrbitalAngularMomentum',
            AngularMomentumExpression(PNOrder=PNOrbitalEvolutionOrder),
            datatype=ellHat.datatype)
        CodeConstructor.AddDependencies(PrecessionVelocities)
        print(CodeConstructor.Atoms)
        # Start the class, write the declarations, initializations, etc.
        f.write("""
class TaylorTn_{PNOrbitalEvolutionOrderString}PN_Q : 
    def TaylorTn_{PNOrbitalEvolutionOrderString}PN_Q({InputArguments}, t_PN1=False, t_PN2=False, StepsPerOrbit=32, ForwardInTime=True, tol=1e-12, MinStep=1e-7) :
        global v, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y
        global rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, Phi
        global terminal1,terminal2,NotForward
        terminal1=True
        terminal2=True
        NotForward=True
{Initializations}
        Phi=0.0
        EvolveSpin1=dot(S_chi1,S_chi1).norm()>1e-12
        EvolveSpin2=dot(S_chi2,S_chi2).norm()>1e-12

        def Recalculate(t, y):
            global v, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y
            global rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, Phi
            v = y[0]
            rfrak_chi1_x = y[1]
            rfrak_chi1_y = y[2]
            rfrak_chi2_x = y[3]
            rfrak_chi2_y = y[4]
            rfrak_frame_x = y[5]
            rfrak_frame_y = y[6]
            rfrak_frame_z = y[7]
            Phi = y[8]
{Evaluations}

{MemberFunctions}

        def FrameFromAngularVelocity_2D_Integrand(rfrak_x, rfrak_y, Omega):
            rfrakMag = np.sqrt(rfrak_x*rfrak_x+rfrak_y*rfrak_y)
            rfrakDot_x = Omega[0]/2.0
            rfrakDot_y = Omega[1]/2.0
            if abs(np.sin(rfrakMag)) > 1e-12 and abs(np.cos(rfrakMag)) > 1e-12:
                omega_v = (Omega[0]*(-rfrak_y/rfrakMag)+Omega[1]*(rfrak_x/rfrakMag))*np.tan(rfrakMag)-Omega[2]
                Omega[0] += -omega_v*np.sin(2*rfrakMag)*(-rfrak_y/rfrakMag)
                Omega[1] += -omega_v*np.sin(2*rfrakMag)*(rfrak_x/rfrakMag)
                Omega[2] +=  omega_v*np.cos(2*rfrakMag)
                dotTerm = (rfrak_x*Omega[0]+rfrak_y*Omega[1])/(rfrakMag*rfrakMag)
                cotTerm = rfrakMag/(2*np.tan(rfrakMag))
                rfrakDot_x = (Omega[0] - rfrak_x*dotTerm)*cotTerm + rfrak_x*dotTerm/2. - 0.5*Omega[2]*rfrak_y
                rfrakDot_y = (Omega[1] - rfrak_y*dotTerm)*cotTerm + rfrak_y*dotTerm/2. + 0.5*Omega[2]*rfrak_x
            return rfrakDot_x, rfrakDot_y
            
""".format(PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString,
           InputArguments=CodeConstructor.CppInputArguments(11),
           Declarations=CodeConstructor.CppDeclarations(2),
           Initializations=CodeConstructor.CppInitializations(4),
           Evaluations=CodeConstructor.CppEvaluations(4, True),
           MemberFunctions=CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities).CppExpressionsAsFunctions(2)))

        # Write the external interfaces to the ODE stepper
        for n,Expressions in zip([1], [T1Expressions]):
            f.write("""
        #Evolve PN
        def TaylorT{n}(t, y):
            global NotForward
            Recalculate(t, y)
            if v>=1.0 and NotForward:
                print("Beyond domain of PN validity, this is a good way to terminate.")
                global terminal1
                terminal1=False
{Computations}
            if dvdt_T{n}<1.0e-12 and NotForward:
                print("v is decreasing, which is not an uncommon way to stop.")
                global terminal2
                terminal2=False
            return CommonRHS(dvdt_T{n}, y)
""".format(n=n,
           Evaluations=CodeConstructor.CppEvaluations(4),
           Computations=CodeOutput.CodeConstructor(PNVariables, Expressions).CppEvaluateExpressions() ))

        # Finish up, writing the common RHS function and closing the class
        f.write("""
        def CommonRHS(dvdt, y):
            dydt=np.zeros(9)
            rfrak_frame=np.zeros(3)
            rfrak_frame[0] = y[5]
            rfrak_frame[1] = y[6]
            rfrak_frame[2] = y[7]
            rfrakdot_frame = quaternion.quaternion_time_series.frame_from_angular_velocity_integrand(rfrak_frame,\\
                quaternion.as_float_array(OmegaVec())[1:])
            dydt[0] = dvdt
            if(EvolveSpin1):
                dydt[1], dydt[2]= FrameFromAngularVelocity_2D_Integrand(y[1], y[2],\\
                    quaternion.as_float_array(S_chi1.inverse()*OmegaVec_chiVec_1()*S_chi1)[1:])
            else:
                dydt[1] = 0.0
                dydt[2] = 0.0
            if(EvolveSpin2):
                dydt[3], dydt[4] = FrameFromAngularVelocity_2D_Integrand(y[3], y[4],\\
                    quaternion.as_float_array(S_chi2.inverse()*OmegaVec_chiVec_2()*S_chi2)[1:])
            else:
                dydt[3] = 0.0
                dydt[4] = 0.0
            dydt[5] = rfrakdot_frame[0]
            dydt[6] = rfrakdot_frame[1]
            dydt[7] = rfrakdot_frame[2]
            dydt[8] = v*v*v/M

            return dydt

        # Prepare for integration
        def terminate(t,y):
            return 1.0*terminal1*terminal2
        terminate.terminal=True
        TMerger=5.0/(256.0*nu*v_i**8)
        TEnd=TMerger
        if t_PN2:
            TEnd=t_PN2
        time=[0.0]
        while time[-1]<TEnd and 2*M*(256*nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit>MinStep:
            time.append(time[-1]+2*M*(256*nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit)
        time=np.delete(time, -1)
        
        # Integrate
        yy=solve_ivp(TaylorT{n}, [time[0],time[-1]], [v_i,rfrak_chi1_x_i,\\
            rfrak_chi1_y_i,rfrak_chi2_x_i,rfrak_chi2_y_i,rfrak_frame_x_i,\\
            rfrak_frame_y_i,rfrak_frame_z_i,0.0], method='DOP853',\\
            t_eval=time, dense_output=True, events=terminate, rtol=tol, atol=tol)           
        if ForwardInTime:
            NotForward=False
            time=[0.0]
            TStart=-3*TMerger
            if t_PN1:
                TStart=t_PN1
            while time[-1]>TStart:
                time.append(time[-1]-2*M*(256*nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit)
            yyForward=solve_ivp(TaylorT{n}, [time[0],time[-1]], [v_i,rfrak_chi1_x_i,\\
                rfrak_chi1_y_i,rfrak_chi2_x_i,rfrak_chi2_y_i,rfrak_frame_x_i,\\
                rfrak_frame_y_i,rfrak_frame_z_i,0.0], method='DOP853',\\
                t_eval=time, dense_output=True, rtol=tol, atol=tol)
            yy.t=np.append(yyForward.t[1:][::-1],yy.t)
            data=np.empty((9,len(yy.t)))
            for i in range(9):
                data[i]=np.append(yyForward.y[i][1:][::-1],yy.y[i])
            yy.y=data
            
        return yy
""".format(n=n,
           Evaluations=CodeConstructor.CppEvaluations(4),
           MemberFunctions=indent(CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities).CppExpressionsAsFunctions(2),'    '),
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString))  


        
        
        
        
# Calculate waveform modes
InputArgumentsEvolution=CodeConstructor.CppInputArguments(11)
execnotebook('../PNTerms/WaveformModes.ipynb')
WaveformModeTerms = [WaveformModes_NoSpin, WaveformModes_Spin_Symm, WaveformModes_Spin_Asymm]
for Term in WaveformModeTerms:
    PNVariables.update(Term)
# For some reason I have to both overwrite these variables and pop them...
PNVariables.AddVariable('nHat', constant=True, fundamental=True,
                 substitution_atoms=[], substitution='Quaternions::xHat', datatype='Quaternions.Quaternion');
PNVariables.AddVariable('lambdaHat', constant=True, fundamental=True,
                substitution_atoms=[], substitution='Quaternions::yHat', datatype='Quaternions.Quaternion');
PNVariables.AddVariable('ellHat', constant=True, fundamental=True,
                substitution_atoms=[], substitution='Quaternions::zHat', datatype='Quaternions.Quaternion');
PNVariables.pop(nHat);
PNVariables.pop(lambdaHat);
PNVariables.pop(ellHat);

PNVariables.AddBasicVariables('chiVec1,chiVec2', datatype='quaternion.quaternion')

PNVariables.AddDerivedVariable('chi1_n', substitution_atoms=[chiVec1,nHat], substitution='chiVec1[1]');
PNVariables.AddDerivedVariable('chi1_lambda', substitution_atoms=[chiVec1,lambdaHat], substitution='chiVec1[2]');
PNVariables.AddDerivedVariable('chi1_ell', substitution_atoms=[chiVec1,ellHat], substitution='chiVec1[3]');
PNVariables.AddDerivedVariable('chi2_n', substitution_atoms=[chiVec2,nHat], substitution='chiVec2[1]');
PNVariables.AddDerivedVariable('chi2_lambda', substitution_atoms=[chiVec2,lambdaHat], substitution='chiVec2[2]');
PNVariables.AddDerivedVariable('chi2_ell', substitution_atoms=[chiVec2,ellHat], substitution='chiVec2[3]');

with open('PNWaveformModes.py', 'w') as f :
    f.write("""# File produced automatically by PNCodeGen.ipynb
import numpy as np
from numpy import conjugate, dot, exp, log, sqrt, pi
from numpy import euler_gamma as EulerGamma
import quaternion
""")

    for PNWaveformModeOrder in [frac(n,2) for n in range(0,8)]:
        print("Working on {0} PN...".format(PNWaveformModeOrder))
        PNWaveformModeOrderString = str(N(PNWaveformModeOrder,2)).replace('.','p')
        ModeExpressions = PNCollection()        
        LM = [[ell,m] for ell in range(2,ellMax+1) for m in range(-ell,ell+1)]
        lenLM=len(LM)
        Evaluations = ['            Modes=np.empty({0}, dtype=complex)'.format(len(LM))]
        for ell in range(2, ellMax+1):
            for m in range(0, ell+1):
                Symm = (SymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
                Asymm = (AsymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
                code1 = "            Symm = {0}".format(sympy.pycode(Symm))
                code2 = "            Asymm = {0}".format(sympy.pycode(Asymm))
                code3 = "            Modes[{2}] = Symm + Asymm".format(ell, m, LM.index([ell,m]))
                code4 = "            Modes[{2}] = {3}conjugate(Symm - Asymm)".format(ell, -m, LM.index([ell,-m]),
                                                                               ('' if ((ell%2)==0) else '-'))
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Symm'.format(ell,m),
                                                   Symm, datatype='double')
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Asymm'.format(ell,m),
                                                   Asymm, datatype='double')
                Evaluations.append("            # (ell, m) = ({0}, +/- {1})".format(ell, m))
                Evaluations.append(code1)
                Evaluations.append(code2)
                Evaluations.append(code3)
                if m!=0:
                    Evaluations.append(code4)
        CodeConstructor = CodeOutput.CodeConstructor(PNVariables, ModeExpressions)
        if(PNWaveformModeOrder>0.9):
            Updates = """            chiVec1 = [0., chi1_n, chi1_lambda, chi1_ell]
            chiVec2 = [0., chi2_n, chi2_lambda, chi2_ell]
"""
        else:
            Updates = ""
        # Start the class, write the declarations, initializations, etc.
        f.write("""
class WaveformModes_{PNWaveformModeOrderString}PN :
    def WaveformModes_{PNWaveformModeOrderString}PN({InputArgumentsEvolution},y) :            
        ModeData=np.empty((len(y[0]),{lenLM}), dtype=complex)
        def WaveformModes({InputArguments}):
            I=1j
{Initializations}
            
{Updates}

{Evaluations}

{Computations}

            return Modes
            
        for i in range(len(y[0])):
            v = y[0,i]
            rfrak_chi1_x = y[1,i]
            rfrak_chi1_y = y[2,i]
            rfrak_chi2_x = y[3,i]
            rfrak_chi2_y = y[4,i]
            rfrak_frame_x = y[5,i]
            rfrak_frame_y = y[6,i]
            rfrak_frame_z = y[7,i]
            Phi = y[8,i]
            xHat=xHat_i
            yHat=yHat_i
            zHat=zHat_i
            M1=M1_i
            M2=M2_i 
            S_chi1=S_chi1_i
            S_chi2=S_chi2_i
            M=M1 + M2
            delta=(M1 - M2)/M
            nu=M1*M2/M**2
            R=exp(rfrak_frame_x*xHat + rfrak_frame_y*yHat + rfrak_frame_z*zHat)
            nHat=R*xHat*conjugate(R)
            lambdaHat=R*yHat*conjugate(R)
            ellHat=R*zHat*conjugate(R)
            R_S1=exp(rfrak_chi1_x*xHat + rfrak_chi1_y*yHat)
            R_S2=exp(rfrak_chi2_x*xHat + rfrak_chi2_y*yHat)
            chiVec1=S_chi1*R_S1*zHat*conjugate(R_S1)*conjugate(S_chi1)
            chiVec2=S_chi2*R_S2*zHat*conjugate(R_S2)*conjugate(S_chi2)
            v_i=v
            rfrak_frame_x_i=rfrak_frame_x
            rfrak_frame_y_i=rfrak_frame_y
            rfrak_frame_z_i=rfrak_frame_z
            chiVec1_i=quaternion.as_float_array(chiVec1)
            chiVec2_i=quaternion.as_float_array(chiVec2)
            ModeData[i,:]=WaveformModes({InputArguments})
        return ModeData
""".format(PNWaveformModeOrderString=PNWaveformModeOrderString,
           InputArgumentsEvolution=InputArgumentsEvolution,
           InputArguments=CodeConstructor.CppInputArguments(22),
           Declarations=CodeConstructor.CppDeclarations(2),
           Initializations=indent(CodeConstructor.CppInitializations(4),'    '),
           Updates=Updates,
           Evaluations=CodeConstructor.CppEvaluations(4),
           MemberFunctions=CodeConstructor.CppExpressionsAsFunctions(2),
           Computations='\n'.join(Evaluations),
           lenLM=lenLM))

print("All done")

Working on 0 PN...
[xHat, yHat, zHat, M1, M2, v, S_chi1, S_chi2, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y, rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, M, delta, nu, R, nHat, ellHat, R_S1, R_S2, chiVec1, chiVec2, chi1chi1, chi1chi2, chi2chi2, chi1_n, chi1_ell, chi2_n, chi2_ell, S_ell, S_n, Sigma_ell, Sigma_n, chi_s_ell, chi_a_ell, Fcal_coeff, Fcal_0, E_0]
Working on 1/2 PN...
[xHat, yHat, zHat, M1, M2, v, S_chi1, S_chi2, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y, rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, M, delta, nu, R, nHat, ellHat, R_S1, R_S2, chiVec1, chiVec2, chi1chi1, chi1chi2, chi2chi2, chi1_n, chi1_ell, chi2_n, chi2_ell, S_ell, S_n, Sigma_ell, Sigma_n, chi_s_ell, chi_a_ell, Fcal_coeff, Fcal_0, E_0]
Working on 1 PN...
[xHat, yHat, zHat, M1, M2, v, S_chi1, S_chi2, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y, rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, M, delta, nu, R, nHat, ellHat, R_S1, R_S2, chiVec1, chiVec2, chi1chi1, chi1chi2, chi2chi

Working on 0 PN...
Working on 1/2 PN...
Working on 1 PN...
Working on 3/2 PN...
Working on 2 PN...
Working on 5/2 PN...
Working on 3 PN...
Working on 7/2 PN...
All done
